# Project imports

In [2]:
"""
All needed imports included here
"""
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch
import pytorch_lightning as pl


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


# Data Loading step

In [3]:
"""
Create data loaders and augmentations needed here
"""
from Data.ShapeNetDataLoader import ShapeNetVoxelData
from Utils.visualization import visualize_occupancy

overfit = True

shapenet_core_path = Path("D:\ShapeNetCore.v2")
shapenet_splits_csv_path = Path("Data/shapenet_splits.csv")
train_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="train", overfit=overfit)
val_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="val", overfit=overfit)
test_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="test", overfit=overfit)

train_sample = test_data[7]
print(f'Voxel Dimensions: {train_sample.shape}')
print(f"Test Set Size: {len(train_data)}")

visualize_occupancy(train_sample.squeeze(), flip_axes=True)

Voxel Dimensions: (1, 128, 128, 128)
Test Set Size: 16


Output()

# Reconstruction Networks

In [12]:
"""
creation, training, and testing of the image2mesh reconstruction networks
"""
from Networks.Image2Mesh import Image2Voxel

model = Image2Voxel(dataloader,dataloader,dataloader,device)

X = torch.rand(10,3,32,32)
pred = model(X)

NameError: name 'dataloader' is not defined

In [7]:
trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else None,
    profiler=True,
    progress_bar_refresh_rate=15
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 67.1 M


Epoch 0:   0%|          | 0/4 [00:00<?, ?it/s]                        

F:\Python\Anaconda\envs\ml3d\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


Epoch 14: 100%|██████████| 4/4 [00:13<00:00,  3.46s/it, loss=0.483, v_num=16]



Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_fit_start        	|  0.0            	|  0.0            
on_validation_start 	|  0.0019375      	|  0.031          
on_validation_epoch_start	|  0.0            	|  0.0            
on_validation_batch_start	|  0.0            	|  0.0            
validation_step_end 	|  0.0            	|  0.0            
on_validation_batch_end	|  0.0            	|  0.0            
on_validation_epoch_end	|  0.0            	|  0.0            
on_validation_end   	|  5.8144         	|  93.03          
on_train_start      	|  0.016          	|  0.016          
on_epoch_start      	|  0.0010667      	|  0.016          
on_train_epoch_start	|  0.0            	|  0.0            
get_train_batch     	|  0.0037         	|  0.111          
on_batch_start      	|  0.0            	|  0.0            
on_train_batch_start	|  0.0            	|  0.0            
training_ste

1

# Purifying predicted Meshes

In [70]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

'\nCode to purify meshes predicted by the previous networks to be used in the retrieval step\n'

# Mesh Encoding

In [19]:
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""
from Networks.VoxelAutoencoder import VoxelAutoencoder
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

torch.cuda.empty_cache()
dim = 128
model = VoxelAutoencoder(dim, train_data, val_data, test_data, device)

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = pl.Trainer(
    max_epochs=64,
    gpus=1 if torch.cuda.is_available() else None,
    log_every_n_steps=1,
    progress_bar_refresh_rate=1,
    logger=logger,
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 19.6 M
----------------------------------
19.6 M    Trainable params
0         Non-trainable params
19.6 M    Total params
78.470    Total estimated model params size (MB)


Epoch 47: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s, loss=0.365, v_num=152]

In [18]:
train_test_sample = train_data[0]

visualize_occupancy(train_test_sample.squeeze(), flip_axes=True)

tensor_test = torch.from_numpy(train_test_sample[np.newaxis, :])

model.eval()
with torch.no_grad():
    decoded_test = model(tensor_test)

tmp_decoded = decoded_test.clone()
tmp_decoded[decoded_test<0.5] = 0
tmp_decoded[decoded_test>=0.5] = 1

decoded_test_np = tmp_decoded.squeeze().detach().numpy()

visualize_occupancy(decoded_test_np, flip_axes=True)

Output()

Output()

# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....